In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from p8_liveness_model import LivenessNet
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os
import tensorflow as tf
# Checking if GPU available
physical_devices = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
dataset = 'live_dataset'
model_path = 'liveness.model'
plot = 'plot.png'
le1 = 'le.pickle'

# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-4
BS = 8
EPOCHS = 50
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(dataset))
data = []
labels = []
# loop over all image paths
for imagePath in imagePaths:
    # extract the class label from the filename, load the image and
    # resize it to be a fixed 32x32 pixels, ignoring aspect ratio
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (32, 32))
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
# convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0


[INFO] loading images...


In [3]:
# encode the labels (which are currently strings) as integers and then
# one-hot encode them
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 2)
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=0.25, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
    width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
    horizontal_flip=True, fill_mode="nearest")
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model = LivenessNet.build(width=32, height=32, depth=3,
    classes=len(le.classes_))
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])
# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
model.summary()

[INFO] compiling model...
[INFO] training network for 50 epochs...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 16)        448       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 16)        2320      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 16)        64        
_______________________________________________________

In [4]:
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS)

# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
    predictions.argmax(axis=1), target_names=le.classes_))
# save the network to disk
print("[INFO] serializing network to '{}'...".format(model))
model.save(model_path, save_format="h5")
# save the label encoder to disk
f = open(le1, "wb")
f.write(pickle.dumps(le))
f.close()


Epoch 1/50
9/9 [==============================] - 1s 67ms/step - loss: 1.1396 - accuracy: 0.5139 - val_loss: 0.6757 - val_accuracy: 0.6000
Epoch 2/50
9/9 [==============================] - 0s 12ms/step - loss: 0.8125 - accuracy: 0.6944 - val_loss: 0.6675 - val_accuracy: 0.6000
Epoch 3/50
9/9 [==============================] - 0s 13ms/step - loss: 0.9139 - accuracy: 0.6806 - val_loss: 0.6718 - val_accuracy: 0.6000
Epoch 4/50
9/9 [==============================] - 0s 13ms/step - loss: 0.8011 - accuracy: 0.7083 - val_loss: 0.6784 - val_accuracy: 0.6000
Epoch 5/50
9/9 [==============================] - 0s 13ms/step - loss: 0.5245 - accuracy: 0.8056 - val_loss: 0.6868 - val_accuracy: 0.6000
Epoch 6/50
9/9 [==============================] - 0s 15ms/step - loss: 0.6659 - accuracy: 0.7222 - val_loss: 0.6988 - val_accuracy: 0.6000
Epoch 7/50
9/9 [==============================] - 0s 15ms/step - loss: 0.4626 - accuracy: 0.7639 - val_loss: 0.7032 - val_accuracy: 0.6000
Epoch 8/50
9/9 [===========

In [5]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plot)